In [ ]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages graphframes:graphframes:0.8.2-spark3.2-s_2.12 pyspark-shell'

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql import functions as F
from pyspark.sql.dataframe import DataFrame, Column

from helper import *

# Spark Configuration

In [ ]:
spark = (
    SparkSession
        .builder
        .appName("graphs")
        .master("local[4]")
        .config("spark.sql.warehouse.dir", "/tmp/warehouse")
        .getOrCreate()
)
sc = spark.sparkContext
sc.setCheckpointDir("/tmp/plan/checkpoint")

In [ ]:
def sparkify(graph: Graph) -> Tuple[DataFrame, DataFrame]:
    edges_df = (
        spark
            .createDataFrame(graph.edges)
            .toDF("src", "dst", "weight")
    )
    vertices_df = (
        spark
            .createDataFrame(graph.vertices)
            .toDF("id")
    )
    return edges_df, vertices_df

# Connected Components

In [ ]:
def bidirectional(edges: DataFrame) -> DataFrame:
    return edges.union(edges.select("dst", "src", "weight"))

## Pregel

### Implementation

In [ ]:
def pregel_cc(edges: DataFrame, values: DataFrame, steps: int) -> DataFrame:
    result_df = values

    for i in range(1, steps + 1):
        result_df = (
            edges.join(result_df, col("src") == col("id"))
                .select(col("dst").alias("id"), col("value").alias("message"))
                .groupBy(col("id")).agg(F.min("message").alias("message"))
                .join(result_df, "id", "right")
                .select("id", F.least("message", "value").alias("value"))
        )
        if i % 5 == 0:
            result_df = result_df.checkpoint()

    return result_df

### Testing

#### Small Graph

![](https://habrastorage.org/webt/yz/gg/jh/yzggjhrflf_8ey-htgoixqbhvng.png)

In [ ]:
edges_df, vertices_df = sparkify(small_graph())

In [ ]:
edges_df.printSchema()

In [ ]:
vertices_df.printSchema()

A directed Graph will not propagate all the messages properly, because the messages won't go back to the src vertices, which prevent the connected components algorithm from the correct evaluation, so each edges should be accompanied with a revese edge:

In [ ]:
edges_df = bidirectional(edges_df)

In [ ]:
pregel_cc_df = vertices_df.withColumn("value", col("id"))
pregel_cc_df = pregel_cc(edges_df, pregel_cc_df, 5)

In [ ]:
pregel_cc_df.show()

#### Large Graph

The large graph is just a bunch of disjoint subgraphs that are essentially linked lists with occasional edges between two vertices in the list

![](https://habrastorage.org/webt/fh/uq/as/fhuqasf_isduq9_mupsga5i4k34.png)

In [ ]:
edges_df, vertices_df = sparkify(graph())

In [ ]:
edges_df.printSchema()

In [ ]:
vertices_df.printSchema()

In [ ]:
edges_df = bidirectional(edges_df)

In [ ]:
pregel_cc_df = vertices_df.withColumn("value", col("id"))

In [ ]:
pregel_cc_df = timer(lambda: pregel_cc(edges_df, pregel_cc_df, 15))

The `cc_df` DataFrame is large, so only the total number of connected components is displayed

In [ ]:
pregel_cc_df.groupBy("value").count().count()

# Single Source Shortest Path

In [ ]:
def get_min_dist_init_df(values: DataFrame, start: int = 1) -> DataFrame:
    return values.withColumn("dist",
        F.when(
            col("id") == F.lit(start),
            F.lit(0)
        )
        .otherwise(F.lit(sys.maxsize // 2))
    )

## Pregel

### Implementation

In [ ]:
def pregel_min_dist(edges: DataFrame, values: DataFrame, steps: int) -> DataFrame:
    result_df = values

    for i in range(1, steps + 1):
        result_df = (
            edges.join(result_df, col("src") == col("id"))
                .select(col("dst").alias("id"), F.expr("dist + weight").alias("message"))
                .groupBy(col("id")).agg(F.min("message").alias("message"))
                .join(result_df, "id", "right")
                .select("id", F.least("message", "dist").alias("dist"))
        )
        if i % 5 == 0:
            result_df = result_df.checkpoint()

    return result_df

### Testing

#### Small Graph

![](https://habrastorage.org/webt/yz/gg/jh/yzggjhrflf_8ey-htgoixqbhvng.png)

In [ ]:
edges_df, vertices_df = sparkify(small_graph())

In [ ]:
vertices_df.printSchema()

In [ ]:
edges_df.printSchema()

In [ ]:
min_dist_df = get_min_dist_init_df(vertices_df, start=1)
min_dist_df = pregel_min_dist(edges_df, min_dist_df, 5)

In [ ]:
min_dist_df.show()

#### Large Graph

The large graph is just a bunch of disjoint subgraphs that are essentially linked lists with occasional edges between two vertices in the list

![](https://habrastorage.org/webt/fh/uq/as/fhuqasf_isduq9_mupsga5i4k34.png)

In [ ]:
disjoint_set_vertices=500
edges_df, vertices_df = sparkify(graph(disjoint_set_vertices))

In [ ]:
vertices_df.printSchema()

In [ ]:
edges_df.printSchema()

In [ ]:
min_dist_df = get_min_dist_init_df(vertices_df, start=1)
min_dist_df = timer(lambda: pregel_min_dist(edges_df, min_dist_df, 12))

All the vertices from the same disjoint set as the vertex `1` should have a distance less that `INF (sys.maxsize)`

In [ ]:
df = min_dist_df.where(col("id") < disjoint_set_vertices).where(col("dist") < sys.maxsize)
timer(lambda: df.count())

All the vertices from the other disjoint sets comparing to the vertex `1` should have the minimum distance exactly `INF (sys.maxsize)`

In [ ]:
df = min_dist_df.where(col("id") > disjoint_set_vertices).groupby("dist").count()
timer(lambda: df.show())

# Topological Sort

## Pregel

### Implementation

In [ ]:
def pregel_topological_sort(edges: DataFrame, values: DataFrame, steps: int) -> DataFrame:
    result_df = values

    for i in range(steps):
        result_df = (
            edges.join(result_df, col("src") == col("id"))
                .select(col("dst").alias("id"), col("ord").alias("message"))
                .groupBy(col("id")).agg(F.max("message").alias("message"))
                .join(result_df, "id", "right")
                .select("id", F.greatest(F.expr("message + 1"), "ord").alias("ord"))
        )
        if i % 5 == 0:
            result_df = result_df.checkpoint()

    return result_df

### Testing

#### Small Graph

![](https://habrastorage.org/webt/yz/gg/jh/yzggjhrflf_8ey-htgoixqbhvng.png)

In [ ]:
edges_df, vertices_df = sparkify(small_graph())

In [ ]:
vertices_df.printSchema()

In [ ]:
edges_df.printSchema()

In [ ]:
edges_df = edges_df.where("src != dst")

In [ ]:
sorted_df = vertices_df.withColumn("ord", F.lit(1))
sorted_df = pregel_topological_sort(edges_df, sorted_df, 5)

In [ ]:
sorted_df.orderBy("ord").show()

#### Large Graph

The large graph is just a bunch of disjoint subgraphs that are essentially linked lists with occasional edges between two vertices in the list

![](https://habrastorage.org/webt/fh/uq/as/fhuqasf_isduq9_mupsga5i4k34.png)

In [ ]:
disjoint_set_vertices=50
edges_df, vertices_df = sparkify(to_dag(graph(disjoint_set_vertices=disjoint_set_vertices)))

In [ ]:
vertices_df.printSchema()

In [ ]:
edges_df.printSchema()

In [ ]:
sorted_df = vertices_df.withColumn("ord", F.lit(1))
sorted_df = timer(lambda: pregel_topological_sort(edges_df, sorted_df, disjoint_set_vertices))

The total number of disjoint sets is equal to 1000 (defautl) and the total number of vertices in each disjoint set is equal to `disjoint_set_vertices` (50), so there should be:
- exactly `1000` vertices for each `ord` number,
- exactly `disjoint_set_vertices` (`50`) unique `ord` numbers.

In [ ]:
df = sorted_df.groupBy("ord").count().orderBy(col("count").desc(), col("ord"))
timer(lambda: df.show(disjoint_set_vertices))

# General Algorithm

## Pregel

In [ ]:
def pregel_superstep(
    edges: DataFrame,
    values: DataFrame,
    message: Column,
    combiner: Callable[[Column], Column],
    computer: Column,
    **columns: Column
) -> DataFrame:
    message_box_df = (
        edges.where(col("src") != col("dst"))
            .join(values, col("src") == col("id"))
            .select(col("dst").alias("id"), message.alias("message"))
    )
    accumulator_df = (
        message_box_df
            .groupby(col("id"))
            .agg(combiner(col("message")).alias("message"))
    )
    return (
        accumulator_df
            .join(values, "id", "right")
            .select("id", computer.alias("value"), *columns.values())
    )

In [ ]:
edges_df, vertices_df = sparkify(small_graph())

![](https://habrastorage.org/webt/yz/gg/jh/yzggjhrflf_8ey-htgoixqbhvng.png)

### Connected Components

In [ ]:
cc_df = vertices_df.withColumn("value", col("id"))

for _ in range(5):
    cc_df = pregel_superstep(
        edges=bidirectional(edges_df),
        values=cc_df,
        message=col("value"),
        combiner=F.min,
        computer=F.least(col("value"), col("message"))
    )

cc_df.show()

### Single Point Shortest Path

In [ ]:
pregel_min_dist_df = get_min_dist_init_df(vertices_df, start=1).withColumnRenamed("dist", "value")

for _ in range(5):
    pregel_min_dist_df = pregel_superstep(
        edges=edges_df,
        values=pregel_min_dist_df,
        message=F.expr("value + weight"),
        combiner=F.min,
        computer=F.least("message", "value")
    )

pregel_min_dist_df.show()

### Topological Sort

In [ ]:
pregel_sort_df = vertices_df.withColumn("value", F.lit(1))

for _ in range(5):
    pregel_sort_df = pregel_superstep(
        edges=edges_df.where("src < dst"),
        values=pregel_sort_df,
        message=col("value"),
        combiner=F.max,
        computer=F.greatest(F.expr("message + 1"), "value")
    )

pregel_sort_df.orderBy("value").show()

### PageRank

Calculate the out degree of each vertex:

In [ ]:
out_deg = (
    edges_df
        .groupBy("src")
        .agg(F.count("src").alias("out_deg"))
        .withColumnRenamed("src", "id")
)

Enrich the `vertices_df` DataFrame with the out degree values. The PageRank algorighm uses the out degree in the denominator, therefore it can't be 0, so I set it to `1` when it's `0` or `null`:

In [ ]:
deg_vert_df = (
    vertices_df
        .join(out_deg, "id", "left")
        .select(
            "id",
            F.coalesce("out_deg", F.lit(1)).alias("out_deg")
        )
)

Initially all the vertices are equal, so the rank value of all of them is set to 1:

In [ ]:
page_rank_result_df = deg_vert_df.withColumn("value", F.lit(1))

In [ ]:
for i in range(10):
    page_rank_result_df = pregel_superstep(
        edges=edges_df,
        values=page_rank_result_df,
        message=col("value"),
        combiner=F.sum,
        computer=F.expr("(0.15 + 0.85 * nvl(message, value)) / out_deg"),
        out_deg=col("out_deg")
    )
    if i % 5 == 0:
        page_rank_result_df = page_rank_result_df.checkpoint()

page_rank_result_df.orderBy("id").show()

The values seem correct:

- all the vertices refer to the vertex `5` directly or indirectly, so it has the highest rank;
- no vertices refer to the vertex `1`, so it has the lowest rank;
- the vertex 6 is in it's own connected component and it doesn't participate in the rank computation process, that is why its rank remained unchanged.

# GraphFrames

**NOTICE**: it might not work in Binder. Simply skip to the "**Distrubuted Graph**" chapter below

In [ ]:
from graphframes import GraphFrame
from graphframes.lib import Pregel

In [ ]:
edges_df, vertices_df = sparkify(small_graph())

In [ ]:
gf_graph = GraphFrame(vertices_df, edges_df)

In [ ]:
with_out_deg_df = (
    gf_graph.outDegrees
        .join(vertices_df, "id", "right")
        .select("id", F.expr("nvl(outDegree, 1.0)").alias("outDegree"))
)

In [ ]:
gf_graph = GraphFrame(with_out_deg_df, gf_graph.edges)

In [ ]:
initialMsg = F.lit(1.0)
# afterMsgAgg = (0.15 + 0.85 * nvl(message, value)) / out_deg
afterMsgAgg = (F.lit(0.15) + F.lit(0.85) * F.coalesce(Pregel.msg(), col("rank"))) / col("outDegree")

ranks = (
    gf_graph.pregel
        .setMaxIter(10)
        # withVertexColumn creates a new column that accumulates the result
        .withVertexColumn(
            "rank", # the name of the column to accumulate the rank value
            initialMsg, # initial value of the column
            afterMsgAgg # how to update the column value when an aggregated message is ready
        )
        # sendMsgToDst defines what the message is
        .sendMsgToDst(Pregel.src("rank"))
        # aggMsgs defines the aggregation operation
        .aggMsgs(F.sum(Pregel.msg()))
)

In [ ]:
result = ranks.run()

In [ ]:
result.orderBy("id").show()

# Distributed Graph

In [ ]:
from helper import graph

Apache Spark naturally partitions the graph by vertices (`vertex-cut`): one vertex is present in multiple partitions, which reduces the problems related to the power-law graphs.

In [ ]:
edges_df, _ = sparkify(graph())

The `F.spark_partition_id()` function calculates the partition number the current row resides in, so let's enrich the `edges_df` DataFrame with this value:

In [ ]:
edges_df_with_part = edges_df.withColumn("partition_id", F.spark_partition_id())

The simple query below demonstrates that there are vertices that reside in different partitions:

In [ ]:
"""
select *
  from t1
     , t1 t2
 where t1.part_id != t2.part_id
   and t1.src == t2.src
"""
df = (
    edges_df_with_part.alias("t1")
        .join(edges_df_with_part.alias("t2"), "src")
        .where("t1.partition_id != t2.partition_id")
        .select("src", "t1.partition_id", "t2.partition_id")
)
df.show(5)